# APLICACIONES EN CIENCIAS DE COMPUTACION

## Laboratorio 2: Implementacion metodos de busqueda ciega para el Problema de Busqueda de Rutas en Mapas

La tarea de este laboratorio consiste en implementar y comparar métodos de busqueda ciega para buscar rutas en mapas.


Al final de este notebook debe responder algunas preguntas.

### Clase Mapa

Estructura para almacenar informacion de un mapa. Tiene dos atributos: <b>neighbors</b> (diccionario que contiene las ciudades vecinas de cada ciudad y las distancias para llegar a ellas) y <b>location</b>, diccionario con las coordenadas X,Y de cada ciudad.

In [2]:
class Mapa:
    def __init__(self, neighbors, location):
        self.neighbors = neighbors
        self.location = location

In [3]:
neighbors = {
 'A': [('Z',75), ('T',118), ('S',140)],
 'B': [('F',211), ('P',101), ('G',90), ('U',85)],
 'C': [('D',120), ('R',146), ('P',138)],
 'D': [('M',75), ('C',120)],
 'E': [('H',86)],
 'F': [('S',99), ('B',211)],
 'G': [('B',90)],
 'H': [('U',98), ('E',86)],
 'I': [('N',87), ('V',92)],
 'L': [('T',111), ('M',70)],
 'M': [('L',70), ('D',75)],
 'N': [('I',87)],
 'O': [('Z',71), ('S',151)],
 'P': [('R',97), ('C',138), ('B',101)],
 'R': [('S',80), ('C',146), ('P',97)],
 'S': [('A',140), ('O',151), ('F',99), ('R',80)],
 'T': [('A',118), ('L',111)],
 'U': [('B',85), ('V',142), ('H',98)],
 'V': [('U',142), ('I',92)],
 'Z': [('O',71), ('A',75)]}

location = {
 'A': (91, 492),
 'B': (400, 327),
 'C': (253, 288),
 'D': (165, 299),
 'E': (562, 293),
 'F': (305, 449),
 'G': (375, 270),
 'H': (534, 350),
 'I': (473, 506),
 'L': (165, 379),
 'M': (168, 339),
 'N': (406, 537),
 'O': (131, 571),
 'P': (320, 368),
 'R': (233, 410),
 'S': (207, 457),
 'T': (94, 410),
 'U': (456, 350),
 'V': (509, 444),
 'Z': (108, 531)}

romania = Mapa(neighbors, location)


### Clase <b>SearchProblem</b>

Esta es una clase abstracta para definir problemas de busqueda. Se debe hacer subclases que implementen los metodos de las acciones, resultados, test de objetivo y el costo de camino. Entonces se puede instanciar las subclases y resolverlos con varias funciones de busqueda.

In [4]:
class SearchProblem(object):
    def __init__(self, initial, goal=None):
        """Este constructor especifica el estado inicial y posiblemente el estado(s) objetivo(s),
        La subclase puede añadir mas argumentos."""
        self.initial = initial
        self.goal = goal

    def actions(self, state):
        """Retorna las acciones que pueden ser ejecutadas en el estado dado.
        El resultado es tipicamente una lista."""
        raise NotImplementedError

    def result(self, state, action):
        """Retorna el estado que resulta de ejecutar la accion dada en el estado state.
        La accion debe ser alguna de self.actions(state)."""
        raise NotImplementedError

    def goal_test(self, state):
        """Retorna True si el estado pasado satisface el objetivo."""
        raise NotImplementedError

    def path_cost(self, c, state1, action, state2):
        """Retorna el costo del camino de state2 viniendo de state1 con 
        la accion action, asumiendo un costo c para llegar hasta state1. 
        El metodo por defecto cuesta 1 para cada paso en el camino."""
        return c + 1

    def value(self, state):
        """En problemas de optimizacion, cada estado tiene un valor. Algoritmos
        como Hill-climbing intentan maximizar este valor."""
        raise NotImplementedError

###  <b> Clase MapSearchProblem </b>  
Esta es una subclase de SearchProblem donde se define concretamente el problema de busqueda en mapa. El constructor recibe el estado inicial, objetivo y un mapa. Se necesita completar Actions (acciones disponibles para un estado dado) y path_cost.

In [10]:
class MapSearchProblem(SearchProblem):
    def __init__(self, initial, goal, mapa):
        """El constructor recibe  el estado inicial, el estado objetivo y un mapa (de clase Mapa)"""
        self.initial = initial
        self.goal = goal
        self.map = mapa

    def actions(self, state):
        """Retorna las acciones ejecutables desde ciudad state.
        El resultado es una lista de strings tipo 'goCity'. 
        Por ejemplo, en el mapa de Romania, las acciones desde Arad serian:
         ['goZerind', 'goTimisoara', 'goSibiu']"""
        neighbors = []
        acciones = []
        neighbors = self.map.neighbors[state]
        for neighbor in neighbors:
            message = "go" + neighbor[0]
            acciones.append(message)
        return acciones

    def result(self, state, action):
        """Retorna el estado que resulta de ejecutar la accion dada desde ciudad state.
        La accion debe ser alguna de self.actions(state)
        Por ejemplo, en el mapa de Romania, el resultado de aplicar la accion 'goZerind' 
        desde el estado 'Arad' seria 'Zerind'"""  
        new_state = action[2]
        return new_state
        
    def goal_test(self, state):
        """Retorna True si state es self.goal"""
        return (self.goal == state) 

    def path_cost(self, c, state1, action, state2):
        """Retorna el costo del camino de state2 viniendo de state1 con la accion action 
        El costo del camino para llegar a state1 es c. El costo de la accion debe ser
        extraido de self.map."""
        action_cost = 0;
        dest_states = self.map.neighbors[state1] # estado destino, state2
        for next_state in dest_states:
            if next_state[0] == state2:
                action_cost = next_state[1]
                break
        return c + action_cost;

### Clase <b>Node</b>

Estructura de datos para almacenar la informacion de un nodo en un <b>arbol de busqueda</b>. Contiene información del nodo padre y el estado que representa el nodo. Tambien incluye la accion que nos llevo al presente nodo y el costo total del camino desde el nodo raiz hasta este nodo.

In [11]:
class Node:
    def __init__(self, state, parent=None, action=None, path_cost=0):
        "Crea un nodo de arbol de busqueda, derivado del nodo parent y accion action"
        self.state = state
        self.parent = parent
        self.action = action
        self.path_cost = path_cost
        self.depth = 0
        if parent:
            self.depth = parent.depth + 1

    def expand(self, problem):
        "Devuelve los nodos alcanzables en un paso a partir de este nodo."
        return [self.child_node(problem, action)
                for action in problem.actions(self.state)]

    def child_node(self, problem, action):
        next = problem.result(self.state, action)
        return Node(next, self, action,
                    problem.path_cost(self.path_cost, self.state, action, next))

    def solution(self):
        "Retorna la secuencia de acciones para ir de la raiz a este nodo."
        return [node.action for node in self.path()[1:]]

    def path(self):
        "Retorna una lista de nodos formando un camino de la raiz a este nodo."
        current = self
        path = []
        # En este problema en particular, como solo quiero conocer las acciones que
        # tomo a la hora de transicionar de un nodo a otro, lo que añadiré a mi lista
        # serán las acciones. En caso se quieran los nodos, cambiar
        # path.append(current.action)
        # por
        # path.append(current)
        while current is not None:
            if current.parent is None:
                break
            path.append(current.action)
            current = current.parent
        # Finalmente, invertimos el recorrido
        path.reverse()
        return path
    
    def __eq__(self, other): 
        "Este metodo se ejecuta cuando se compara nodos. Devuelve True cuando los estados son iguales"
        return isinstance(other, Node) and self.state == other.state

### <b> Definición de fronteras</b>
Se definen las clases correspondientes a colas FIFO y LIFO para usar como fronteras.

In [12]:
from collections import deque

class FIFO(deque):
    """Una cola First-In-First-Out"""
    def pop(self):
        return self.popleft()

class LIFO(deque):
    """Una cola Last-In-First-Out"""

import heapq
class PQ:
    heap = []
    def __init(self):
        heapq.heapify(self.heap)
        
    def pop(self):
        return heapq.heappop(self.heap)
    
    def append(self, node):
        heapq.heappush(self.heap, node)
    
    def extend(self, nodes):
        for node in nodes:
            heapq.heappush(self.heap, node)

### <b>Algoritmo general de búsqueda con memoria de nodos expandidos (Graph Search)</b>

Algoritmo de general de busqueda ciega con memoria de estados visitados. El argumento frontier debe ser una cola vacia FIFO o LIFO. Explora la frontera de acuerdo al algoritmo de búsqueda en árboles visto en clase



In [13]:
def graph_search(problem, frontier):
    frontier.append(Node(problem.initial))
    explored = set()     # memoria de estados visitados
    expanded_nodes = 0   # contador de nodos expandidos
    while frontier:
        node = frontier.pop()
        if problem.goal_test(node.state):
            return node, expanded_nodes
        explored.add(node.state)
        expanded_nodes = expanded_nodes + 1
        frontier.extend(child for child in node.expand(problem)
                        if child.state not in explored and
                        child not in frontier)
    return None

### <b> Probando los algoritmos de Busqueda</b> 
Ejecutar la búsqueda BFS, DFS y UCS aplicando la frontera correspondiente e imprimir los resultados. El output de la siguiente celda debe ser el siguiente:

Solucion obtenida con BFS: ['goS', 'goF', 'goB']. Nodos expandidos = 9.

Solucion obtenida con DFS: ['goS', 'goR', 'goP', 'goB']. Nodos expandidos = 4. 

Solucion obtenida con UCS: ['goS', 'goR', 'goP', 'goB']. Nodos expandidos = 12. 


In [14]:
p = MapSearchProblem('A', 'B', romania)   # problema de busqueda de ruta de Arad a Bucharest

# Para un BFS utilizamos una cola
last, expanded_nodes = graph_search(p, FIFO())
print('Solucion obtenida con BFS: ', last.path(), '. Nodos expandidos = ', expanded_nodes, '.')

# Para un DFS utilizamos una pila
last, expanded_nodes = graph_search(p, LIFO())
print('Solucion obtenida con DFS: ', last.path(), '. Nodos expandidos = ', expanded_nodes, '.')

# Para un UCS utilizamos una cola de prioridad
# En la clase PQ, faltaria definirle un iterador para que realice la expansión correctamente, pero todo lo demás
# está bien definido
#last, expanded_nodes = graph_search(p, PQ())
#print('Solucion obtenida con DFS: ', last.path(), '. Nodos expandidos = ', expanded_nodes, '.')

Solucion obtenida con BFS:  ['goS', 'goF', 'goB'] . Nodos expandidos =  9 .
Solucion obtenida con DFS:  ['goS', 'goR', 'goP', 'goB'] . Nodos expandidos =  4 .


## Preguntas

<b>0) Completar el código y ejecutar satisfactoriamente las pruebas</b> (3 pts)





Completado satisfactoriamente.

<b>1) Probar BFS y DFS en los siguientes problemas (registre la ruta tomada y los nodos expandidos): </b> (4 pts)
* p = MapSearchProblem('A', 'D', romania)
* p = MapSearchProblem('F', 'Z', romania)
* p = MapSearchProblem('N', 'R', romania)
* p = MapSearchProblem('A', 'U', romania)
* p = MapSearchProblem('T', 'G', romania)

In [15]:
p = MapSearchProblem('A', 'D', romania)
# Para un BFS utilizamos una cola
last, expanded_nodes = graph_search(p, FIFO())
print('Solucion obtenida con BFS: ', last.path(), '. Nodos expandidos = ', expanded_nodes, '.')
# Para un DFS utilizamos una pila
last, expanded_nodes = graph_search(p, LIFO())
print('Solucion obtenida con DFS: ', last.path(), '. Nodos expandidos = ', expanded_nodes, '.')

Solucion obtenida con BFS:  ['goT', 'goL', 'goM', 'goD'] . Nodos expandidos =  12 .
Solucion obtenida con DFS:  ['goS', 'goR', 'goC', 'goD'] . Nodos expandidos =  13 .


In [16]:
p = MapSearchProblem('F', 'Z', romania)
# Para un BFS utilizamos una cola
last, expanded_nodes = graph_search(p, FIFO())
print('Solucion obtenida con BFS: ', last.path(), '. Nodos expandidos = ', expanded_nodes, '.')
# Para un DFS utilizamos una pila
last, expanded_nodes = graph_search(p, LIFO())
print('Solucion obtenida con DFS: ', last.path(), '. Nodos expandidos = ', expanded_nodes, '.')

Solucion obtenida con BFS:  ['goS', 'goA', 'goZ'] . Nodos expandidos =  9 .
Solucion obtenida con DFS:  ['goB', 'goP', 'goC', 'goD', 'goM', 'goL', 'goT', 'goA', 'goZ'] . Nodos expandidos =  16 .


In [17]:
p = MapSearchProblem('N', 'R', romania)
# Para un BFS utilizamos una cola
last, expanded_nodes = graph_search(p, FIFO())
print('Solucion obtenida con BFS: ', last.path(), '. Nodos expandidos = ', expanded_nodes, '.')
# Para un DFS utilizamos una pila
last, expanded_nodes = graph_search(p, LIFO())
print('Solucion obtenida con DFS: ', last.path(), '. Nodos expandidos = ', expanded_nodes, '.')

Solucion obtenida con BFS:  ['goI', 'goV', 'goU', 'goB', 'goP', 'goR'] . Nodos expandidos =  11 .
Solucion obtenida con DFS:  ['goI', 'goV', 'goU', 'goB', 'goP', 'goR'] . Nodos expandidos =  18 .


In [18]:
p = MapSearchProblem('A', 'U', romania)
# Para un BFS utilizamos una cola
last, expanded_nodes = graph_search(p, FIFO())
print('Solucion obtenida con BFS: ', last.path(), '. Nodos expandidos = ', expanded_nodes, '.')
# Para un DFS utilizamos una pila
last, expanded_nodes = graph_search(p, LIFO())
print('Solucion obtenida con DFS: ', last.path(), '. Nodos expandidos = ', expanded_nodes, '.')

Solucion obtenida con BFS:  ['goS', 'goF', 'goB', 'goU'] . Nodos expandidos =  14 .
Solucion obtenida con DFS:  ['goS', 'goR', 'goP', 'goB', 'goU'] . Nodos expandidos =  5 .


In [19]:
p = MapSearchProblem('T', 'G', romania)
# Para un BFS utilizamos una cola
last, expanded_nodes = graph_search(p, FIFO())
print('Solucion obtenida con BFS: ', last.path(), '. Nodos expandidos = ', expanded_nodes, '.')
# Para un DFS utilizamos una pila
last, expanded_nodes = graph_search(p, LIFO())
print('Solucion obtenida con DFS: ', last.path(), '. Nodos expandidos = ', expanded_nodes, '.')

Solucion obtenida con BFS:  ['goA', 'goS', 'goF', 'goB', 'goG'] . Nodos expandidos =  13 .
Solucion obtenida con DFS:  ['goL', 'goM', 'goD', 'goC', 'goP', 'goB', 'goG'] . Nodos expandidos =  13 .


<b>2) Compare los nodos expandidos al buscar la ruta entre P y R, y explique la diferencia utilizando la teoría detrás de las búsquedas BFS y DFS </b> (4 pts)

In [20]:
p = MapSearchProblem('P', 'R', romania)
# Para un BFS utilizamos una cola
last, expanded_nodes = graph_search(p, FIFO())
print('Solucion obtenida con BFS: ', last.path(), '. Nodos expandidos = ', expanded_nodes, '.')
# Para un DFS utilizamos una pila
last, expanded_nodes = graph_search(p, LIFO())
print('Solucion obtenida con DFS: ', last.path(), '. Nodos expandidos = ', expanded_nodes, '.')

Solucion obtenida con BFS:  ['goR'] . Nodos expandidos =  1 .
Solucion obtenida con DFS:  ['goR'] . Nodos expandidos =  19 .


En el caso de BFS, como nos encontramos a distancia uno y justo exploramos al nodo R en nuestro recorrido del primer nivel, se acabó.
Sin embargo, a pesar de estar a distancia uno, cuando lanzamos el DFS, como realiza una búsqueda a profundidad y al parecer este comienza yendo hacia el estado B, entonces se termina desviando mucho recorriendo a todo lo que B
nos permite explorar, y por mala suerte nuestra, luego cuando vuelve, encuentra a R.

<b>3) Justifique teóricamente la optimalidad (o no-optimalidad) de cada algoritmo</b> (4 pts)

BFS:
Complejidad en tiempo: $O(V + E)$ o $O(b^{d + 1})$.
Complejidad en memoria: $O(V)$ o $O(b^d)$.
Encolamos una sola vez cada nodo ya que estamos memorizando los que ya han sido visitados, y como no lo volvemos a encolar, entonces recorremos a ellos $O(V)$ en tiempo, y como para cada uno analizaré en $O(1)$ a cada vecino, la complejidad total en tiempo sería $O(V + E)$. En memoria, como tendremos nuestra cola en la cual iremos pusheando y popeando a los elementos, tendremos $O(V)$.
La manera análoga expresada es con la cantidad de nodos en la frontera y la cantidad de nodos como la sumatoria de los visitados hasta ahora.

DFS:
Complejidad en tiempo: $O(V + E)$ o $O(b^m)$.
Complejidad en memoria: $O(V)$ o $O(m)$.
Sin pensar tanto, realizamos un backtracking descubriendo todo lo que podemos en nuestro camino.
La manera análoga es con la profundidad de nuestro backtracking en la pila, y en nodos todos los que hemos creado.

UCS:
Complejidad en tiempo: $O(V \log V + E)$  o $O(b^{1 + C / e})$.
Complejidad en memoria: $O(V + E)$ o $O(b^{1 + C / e})$
En UCS, una manera de mejorar la complejidad en memoria sería utilizar un set en vez de una cola de prioridad,
puesto que en una cola de prioridad obtenemos el menor y si hay repetidos y no podemos relajar más una arista,
estaríamos quedándonos con el menor y lo descartaríamos, sin embargo, en un set, reemplazamos un solo valor
y como internamente funciona con un árbol balanceado, el iterador inicial apunta al menor elemento en este set.

<b>4) ¿En qué caso sugeriría usar cada algoritmo?</b> (3 pts)


El algoritmo BFS lo usaría cuando sé que todos los nodos tienen un mismo costo constante (por ejemplo, si todos tuviesen costo 1, estamos computando la distancia mínima de la fuente a cada nodo alcanzado).
El DFS lo utilizaría solamente cuando quiero encontrar cualquier camino o saber si es que están conectados (es decir, si existe al menos un camino que una la fuente con el destino).
El algoritmo UCS lo utilizaría cuando tengo pesos distintos. Al igual que el famoso algoritmo de Dijkstra, la relajación greedy ocurre al tomar el peso de menor costo.

<b>5) ¿Por qué se implementa una memoria de estados visitados en el algoritmo de búsqueda en grafo? </b> (2 pts)

Se implementa una memoria de estados visitados porque sin esta, podríamos estar llegando a nodos que ya han sido analizados en nuestro algoritmo, por lo que podríamos o bien analizar muchos casos más, o en el peor escenario, tener un bucle infinito.